In [1]:
#!/usr/bin/env python

import math
import healpy as hp
from classy import Class
from scipy.stats import norm
import numpy as np
import sys
import time
sys.path.insert(1, '../../../PySSC/')
sys.path.insert(1, '../../../CLASS/')
import PySSC

try:
    import gi

    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())


In [2]:
cosmo = Nc.HICosmoDEXcdm()
cosmo.props.H0  =  67.81
cosmo.props.Omegac  =  0.2612
cosmo.props.Omegab =  0.0486
cosmo.props.Omegax = 1 - 0.0486 -  0.2612
cosmo.props.w = -1.0


reion = Nc.HIReionCamb.new () 
prim = Nc.HIPrimPowerLaw.new () 



cosmo.add_submodel (reion)
cosmo.add_submodel (prim)

dist = Nc.Distance.new (2.0)

tf = Nc.TransferFunc.new_from_name ("NcTransferFuncEH")

psml = Nc.PowspecMLTransfer.new (tf)
psml.require_kmin (1.0e-6)
psml.require_kmax (1.0e3)

psf = Ncm.PowspecFilter.new (psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0 ()
prim.props.n_SA = 0.9660
print(0.82505858,cosmo.sigma8(psf),cosmo.Omega_k0())

old_amplitude = math.exp (prim.props.ln10e10ASA)
prim.props.ln10e10ASA = math.log ((0.82505858/ cosmo.sigma8(psf))**2 * old_amplitude)
print(0.82505858, cosmo.sigma8(psf))

z_obs  = Ncm.Vector.new_array(np.linspace(0.1,0.8,8))
lnM_obs = Ncm.Vector.new_array(np.linspace(np.log(10)*13.0,np.log(10)*15, 2))

0.82505858 0.8900071895091025 -9.085228043836935e-05
0.82505858 0.82505858


# Construct the $S_{ij}$  Matrix

`kernel and masks`

In [3]:
cosmo_fid = Class()
cosmo_fid.set({'h':cosmo.props.H0/100 ,'Omega_cdm':cosmo.props.Omegac ,'Omega_b':cosmo.props.Omegab ,'sigma8':0.82505858,'n_s':0.9660,'output':'mPk'})
cosmo_fid.compute()
nz       = 400
z_arr    = np.linspace(0,1.9,num=nz+1)[1:]


nbins_T   = z_obs.len() -1


kernels_T = np.zeros((nbins_T,nz))
for i in range(nbins_T):
    zminbin = z_obs.get(i) ; zmaxbin = z_obs.get(i+1) ; Dz = zmaxbin-zminbin
    for iz in range(nz):
        z = z_arr[iz]
        if ((z>zminbin) and (z<=zmaxbin)):
            kernels_T[i,iz] = 1/Dz




NSIDE = 2*512
NPIX = hp.nside2npix(NSIDE)
m = np.zeros(hp.nside2npix(NSIDE))
pix_theta_ecl, pix_phi_ecl = hp.pix2ang(NSIDE, np.arange(NPIX))

#Guaranteed mask
mask1_guaranteed = np.zeros(NPIX)
mask2_guaranteed = np.zeros(NPIX)
mask3_guaranteed = np.zeros(NPIX)

mask1_guaranteed_condition = (pix_phi_ecl > 3.*np.pi/4.) & (pix_phi_ecl < 13.*np.pi/12.) & (pix_theta_ecl < np.pi/2. - 30.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 80.*np.pi/180)
mask2_guaranteed_condition = (pix_phi_ecl > 11.*np.pi/6.) & (pix_theta_ecl < np.pi/2. - 20.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 40.*np.pi/180)
mask3_guaranteed_condition = (pix_phi_ecl < np.pi/4.) & (pix_theta_ecl < np.pi/2. - 20.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 40.*np.pi/180)

mask1_guaranteed[mask1_guaranteed_condition] = 1
mask2_guaranteed[mask2_guaranteed_condition] = 1
mask3_guaranteed[mask3_guaranteed_condition] = 1

mask_guaranteed = mask1_guaranteed + mask2_guaranteed + mask3_guaranteed

#Total mask
mask1_full = np.zeros(NPIX)
mask2_full = np.zeros(NPIX)
mask3_full = np.zeros(NPIX)

mask1_full_condition = (pix_phi_ecl > 2.*np.pi/3.) & (pix_phi_ecl < 3.*np.pi/2.) & (pix_theta_ecl < np.pi/2. - 10.*np.pi/180) & (pix_theta_ecl > np.pi/2. - 80.*np.pi/180)
mask2_full_condition = (pix_phi_ecl > 11.*np.pi/6.) & (pix_theta_ecl < np.pi/2.) & (pix_theta_ecl > np.pi/2. - 45.*np.pi/180)
mask3_full_condition = (pix_phi_ecl < np.pi/4.) & (pix_theta_ecl < np.pi/2.) & (pix_theta_ecl > np.pi/2. - 45.*np.pi/180)

mask1_full[mask1_full_condition] = 1
mask2_full[mask2_full_condition] = 1
mask3_full[mask3_full_condition] = 1

mask_full = mask1_full + mask2_full + mask3_full

`S matrices`

In [4]:
#Full sky
t0 = time.time()
S_lacasa = PySSC.Sij(z_arr,kernels_T,cosmo_Class=cosmo_fid)
t1 = time.time()
print('Full sky time (seconds) = ' + str(t1-t0))


S_full = Ncm.Matrix.new(S_lacasa.shape[0],S_lacasa.shape[1])
for i in range(len(S_lacasa)):
    for j in range(len(S_lacasa[i])):
        S_full.set(i,j, S_lacasa[i][j])


#Partial-Sky guaranteed
t0 = time.time()
S_lacasa = PySSC.Sij_psky(z_arr,kernels_T,mask=mask_guaranteed,cosmo_Class=cosmo_fid)
t1 = time.time()
print('Partial sky guaranteed time (seconds) = ' + str(t1-t0))

S_guaranteed = Ncm.Matrix.new(S_lacasa.shape[0],S_lacasa.shape[1])
for i in range(len(S_lacasa)):
    for j in range(len(S_lacasa[i])):
        S_guaranteed.set(i,j, S_lacasa[i][j])


#Partial-Sky maximal
t0 = time.time()
S_lacasa = PySSC.Sij_psky(z_arr,kernels_T,mask=mask_full,cosmo_Class=cosmo_fid)
t1 = time.time()
print('Partial sky maximal time (seconds) = ' + str(t1-t0))

S_maximal = Ncm.Matrix.new(S_lacasa.shape[0],S_lacasa.shape[1])
for i in range(len(S_lacasa)):
    for j in range(len(S_lacasa[i])):
        S_maximal.set(i,j, S_lacasa[i][j])

Full sky time (seconds) = 0.628016471862793
Partial sky guaranteed time (seconds) = 29.146015167236328
Partial sky maximal time (seconds) = 18.937682390213013


# Define proxy modelling

In [5]:
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e14),math.log(10)*np.log10(1e16)))
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (0.25,2))

# Initialize the `ClusterAbundance` object

In [6]:
mulf = Nc.MultiplicityFuncTinker.new ()
mulf.set_mdef (Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta (200.0)

hmf = Nc.HaloMassFunction.new (dist, psf, mulf)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf,hbias_Tinker)

# Data model

$f(x_i)$ for the Gaussian distribution $$-2\ln L = \sum_{ij}\left[(f(x_i) - y_i)C^{-1}_{ij}(f(x_j) - y_j)\right] + \dots$$

In [7]:
mset = Ncm.MSet.new_array([cosmo,cluster_m,cluster_z])


ncounts_gauss = Nc.DataClusterNCountsGauss.new(cad)
ncounts_gauss.set_size((z_obs.len()-1)*(lnM_obs.len()-1))
ncounts_gauss.set_init(True)
ncounts_gauss.use_norma(True)
ncounts_gauss.set_z_obs(z_obs)
ncounts_gauss.set_lnM_obs(lnM_obs)
rng = Ncm.RNG.seeded_new (None, 42)

# Creating the fit object and calculating the MCMC

`No SSC`

`Full-Sky`

In [8]:
print('Full Sky')

ncounts_gauss.set_has_ssc(True)
guaranteed_area = 2959.1
cad.set_area(guaranteed_area*(np.pi/180)**2)
ncounts_gauss.set_s_matrix(S_full)
ncounts_gauss.resample(mset,rng)
ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)


dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()


lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

init_sampler = Ncm.MSetTransKernGauss.new (0)
init_sampler.set_mset (mset)
init_sampler.set_prior_from_mset ()
init_sampler.set_cov_from_rescale (1.0) #1


sampler = 'apes'

nwalkers = int (math.ceil (500)) #500
ssize    = 50000 #1000000

walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())

esmcmc  = Ncm.FitESMCMC.new_funcs_array (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa)


esmcmc.set_data_file ("mcmc_fullsky.fits")

esmcmc.set_nthreads (4)
t0 = time.time()
esmcmc.start_run ()
esmcmc.run (ssize / nwalkers)
#esmcmc.run (10)
#esmcmc.run_lre (50, 1.0e-3)
esmcmc.end_run ()
t1 = time.time()

esmcmc.mean_covar ()
fit.log_covar ()

print('Full sky maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

Full Sky
y: 24472.535 59203.164 99978.740 141638.839 177139.993 205127.541 224353.942
0.82505858
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0004.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 113889135.
# NcmMSetCatalog: Current mean:   3.2631e+05   0.99213      0.2611      -1.0002       3.2628     
# NcmMSetCatalog: Current msd:    35254        0.017079     0.0004457    0.00044527   0.032435   
# NcmMSetCatalog: Current sd:     7.8831e+05   0.3819       0.0099663    0.0099566    0.72527    
# NcmMSetCatalog: Current var:    6.2144e+11   0.14585      9.9326e-05   9.9133e-05   0.52602    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac: -nan         -nan


(process:72054): NUMCOSMO-WARNING **: 17:43:31.975: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 17:43:46.247: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 17:43:48.390: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   1.0501e+05   0.93094      0.26012     -0.99982      3.214      
# NcmMSetCatalog: Current msd:    9417.9       0.0059132    0.00028887   0.00033165   0.011421   
# NcmMSetCatalog: Current sd:     4.2118e+05   0.26445      0.012919     0.014832     0.51078    
# NcmMSetCatalog: Current var:    1.7739e+11   0.069931     0.00016689   0.00021999   0.2609     
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  1.0029       1.0704       1.1242       1.1138       1.0782     
# NcmMSetCatalog: Maximal Shrink factor = 1.14226               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `4':
# NcmMSetCatalog: Constant break point at `1':
# NcmFitESMCMC:acceptance ratio 52.2000% (last update 50.0000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC


(process:72054): NUMCOSMO-WARNING **: 17:44:05.657: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 17:44:14.232: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 17:44:21.578: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 17:44:52.492: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   60533        0.89136      0.25975     -0.99993      3.1539     
# NcmMSetCatalog: Current msd:    5450.8       0.0034926    0.00020122   0.000271     0.006852   
# NcmMSetCatalog: Current sd:     3.2248e+05   0.20662      0.011904     0.016032     0.40537    
# NcmMSetCatalog: Current var:    1.0399e+11   0.042694     0.00014171   0.00025704   0.16432    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99939      1.0275       1.0904       1.117        1.0374     
# NcmMSetCatalog: Maximal Shrink factor = 1.11724               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `7':
# NcmMSetCatalog: Constant break point at `3':
# NcmFitESMCMC:acceptance ratio 49.2667% (last update 36.8000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC

...

# NcmMSetCatalog: Current mean:   5369.2       0.83715      0.25847     -1.0134       3.0601     
# NcmMSetCatalog: Current msd:    478.48       0.00032154   2.7627e-05   0.00027715   0.00072425 
# NcmMSetCatalog: Current sd:     96293        0.064709     0.0055598    0.055775     0.14575    
# NcmMSetCatalog: Current var:    9.2724e+09   0.0041872    3.0912e-05   0.0031108    0.021244   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99976      1.0066       1.0649       1.1034       1.0305     
# NcmMSetCatalog: Maximal Shrink factor = 1.10358               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `81':
# NcmMSetCatalog: Constant break point at `15':
# NcmFitESMCMC:acceptance ratio 19.4100% (last update 31.4000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMC


(process:72054): NUMCOSMO-WARNING **: 18:23:48.657: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 18:23:56.662: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   5242.3       0.83696      0.25845     -1.0137       3.0596     
# NcmMSetCatalog: Current msd:    466.97       0.00031387   2.7032e-05   0.00027167   0.00070734 
# NcmMSetCatalog: Current sd:     95129        0.063941     0.0055069    0.055343     0.1441     
# NcmMSetCatalog: Current var:    9.0496e+09   0.0040884    3.0326e-05   0.0030628    0.020764   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99977      1.0064       1.063        1.0995       1.0296     
# NcmMSetCatalog: Maximal Shrink factor = 1.09968               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `83':
# NcmMSetCatalog: Constant break point at `15':
# NcmFitESMCMC:acceptance ratio 19.8220% (last update 35.6000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMC


(process:72054): NUMCOSMO-WARNING **: 18:35:37.503: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   4368.6       0.83569      0.2583      -1.0153       3.0564     
# NcmMSetCatalog: Current msd:    387.68       0.00026099   2.2941e-05   0.00023354   0.00059045 
# NcmMSetCatalog: Current sd:     86688        0.05836      0.0051298    0.052221     0.13203    
# NcmMSetCatalog: Current var:    7.5148e+09   0.0034058    2.6315e-05   0.0027271    0.017432   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99981      1.0053       1.0491       1.0738       1.0239     
# NcmMSetCatalog: Maximal Shrink factor = 1.07409               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `100':
# NcmMSetCatalog: Constant break point at `65':
# NcmFitESMCMC:acceptance ratio 22.1232% (last update 30.0000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESM

`Partial-Sky guaranteed`

In [9]:
print('Partial sky guaranteed')

guaranteed_area = 2959.1
cad.set_area(guaranteed_area*(np.pi/180)**2)
ncounts_gauss.set_s_matrix(S_guaranteed)
ncounts_gauss.resample(mset,rng)
ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)


dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()


lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

init_sampler = Ncm.MSetTransKernGauss.new (0)
init_sampler.set_mset (mset)
init_sampler.set_prior_from_mset ()
init_sampler.set_cov_from_rescale (1.0) #1


sampler = 'apes'

nwalkers = int (math.ceil (500)) #500
ssize    = 50000 #1000000

walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())

esmcmc  = Ncm.FitESMCMC.new_funcs_array (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa)


esmcmc.set_data_file ("mcmc_partialskyguaranteed.fits")

esmcmc.set_nthreads (4)
t0 = time.time()
esmcmc.start_run ()
esmcmc.run (ssize / nwalkers)
#esmcmc.run (10)
#esmcmc.run_lre (50, 1.0e-3)
esmcmc.end_run ()
t1 = time.time()

esmcmc.mean_covar ()
fit.log_covar ()

print('Full sky maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

Partial sky guaranteed
y: 24090.629 59236.016 100211.349 143398.383 180148.206 207808.669 230816.826
0.835039921498831
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0004.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 2407947344.
# NcmMSetCatalog: Current mean:   58323        0.99564      0.25902     -1.016        3.264      
# NcmMSetCatalog: Current msd:    8798.7       0.017344     0.00045043   0.00043861   0.032981   
# NcmMSetCatalog: Current sd:     1.9674e+05   0.38782      0.010072     0.0098076    0.73747    
# NcmMSetCatalog: Current var:    3.8709e+10   0.1504       0.00010144   9.6188e-05   0.54386    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current s


(process:72054): NUMCOSMO-WARNING **: 18:38:21.710: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   23614        0.94943      0.2579      -1.0165       3.2384     
# NcmMSetCatalog: Current msd:    3031.3       0.0074949    0.00032417   0.00031484   0.014481   
# NcmMSetCatalog: Current sd:     1.174e+05    0.29028      0.012555     0.012194     0.56084    
# NcmMSetCatalog: Current var:    1.3783e+10   0.08426      0.00015763   0.00014869   0.31455    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  1.0032       1.1308       1.1943       1.1551       1.1408     
# NcmMSetCatalog: Maximal Shrink factor = 1.22447               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `3':
# NcmMSetCatalog: Constant break point at `0':
# NcmFitESMCMC:acceptance ratio 54.6000% (last update 56.0000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC


(process:72054): NUMCOSMO-WARNING **: 18:42:41.713: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.

(process:72054): NUMCOSMO-WARNING **: 18:43:07.556: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   10376        0.87948      0.26227     -1.014        3.0964     
# NcmMSetCatalog: Current msd:    1313.3       0.0034557    0.00025696   0.00029075   0.0072082  
# NcmMSetCatalog: Current sd:     77696        0.20444      0.015202     0.017201     0.42644    
# NcmMSetCatalog: Current var:    6.0366e+09   0.041795     0.0002311    0.00029587   0.18186    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99934      1.0284       1.1843       1.2009       1.051      
# NcmMSetCatalog: Maximal Shrink factor = 1.21931               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `7':
# NcmMSetCatalog: Constant break point at `4':
# NcmFitESMCMC:acceptance ratio 40.1333% (last update 12.4000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCMC

...

# NcmMSetCatalog: Current mean:   1047.2       0.84573      0.25607     -1.0278       3.0865     
# NcmMSetCatalog: Current msd:    115.8        0.00035082   7.017e-05    0.0004077    0.0010249  
# NcmMSetCatalog: Current sd:     23161        0.070164     0.014034     0.08154      0.20497    
# NcmMSetCatalog: Current var:    5.3642e+08   0.0049229    0.00019695   0.0066488    0.042013   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99985      1.0178       1.0583       1.0345       1.0478     
# NcmMSetCatalog: Maximal Shrink factor = 1.07051               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `80':
# NcmMSetCatalog: Constant break point at `21':
# NcmFitESMCMC:acceptance ratio 27.1646% (last update 57.8000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMC


(process:72054): NUMCOSMO-WARNING **: 20:22:54.708: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   900.56       0.84612      0.25531     -1.0305       3.0908     
# NcmMSetCatalog: Current msd:    97.531       0.00029679   6.1248e-05   0.00038617   0.00087746 
# NcmMSetCatalog: Current sd:     21256        0.064684     0.013349     0.084164     0.19124    
# NcmMSetCatalog: Current var:    4.5183e+08   0.004184     0.00017818   0.0070835    0.036572   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99988      1.015        1.0466       1.0284       1.0389     
# NcmMSetCatalog: Maximal Shrink factor = 1.0564                
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `95':
# NcmMSetCatalog: Constant break point at `56':
# NcmFitESMCMC:acceptance ratio 31.6128% (last update 60.8000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMC


(process:72054): NUMCOSMO-WARNING **: 20:27:06.514: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   876.61       0.84616      0.25518     -1.0311       3.0914     
# NcmMSetCatalog: Current msd:    94.547       0.00028795   5.9787e-05   0.00038231   0.00085314 
# NcmMSetCatalog: Current sd:     20929        0.06374      0.013234     0.084627     0.18885    
# NcmMSetCatalog: Current var:    4.3802e+08   0.0040628    0.00017515   0.0071617    0.035665   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99988      1.0145       1.0445       1.0272       1.0376     
# NcmMSetCatalog: Maximal Shrink factor = 1.05429               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `98':
# NcmMSetCatalog: Constant break point at `56':
# NcmFitESMCMC:acceptance ratio 32.3670% (last update 52.6000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMC

`Partial Sky maximal`

In [10]:
print('Partial Sky maximal area')

maximal_area = 10009.97
cad.set_area(maximal_area*(np.pi/180)**2)
ncounts_gauss.set_s_matrix(S_maximal)
ncounts_gauss.resample(mset,rng)
ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)


dset = Ncm.Dataset.new()
dset.append_data(ncounts_gauss)


cosmo.props.Omegac_fit = True
cosmo.props.Omegab_fit    = False
cosmo.props.w_fit = True
#cosmo.props.H0_fit = True
prim.props.ln10e10ASA_fit = True
mfunc_oa = Ncm.ObjArray.new ()
mfunc_sigma8 = Ncm.MSetFuncList.new ("NcHICosmo:sigma8", psf)
mfunc_oa.add (mfunc_sigma8) 
print (mfunc_sigma8.eval0 (mset))

mset.prepare_fparam_map ()


lh = Ncm.Likelihood.new (dset)

fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset,  Ncm.FitGradType.NUMDIFF_FORWARD)

init_sampler = Ncm.MSetTransKernGauss.new (0)
init_sampler.set_mset (mset)
init_sampler.set_prior_from_mset ()
init_sampler.set_cov_from_rescale (1.0) #1


sampler = 'apes'

nwalkers = int (math.ceil (500)) #500
ssize    = 50000 #1000000

walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())

esmcmc  = Ncm.FitESMCMC.new_funcs_array (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE, mfunc_oa)


esmcmc.set_data_file ("mcmc_partialskymaximal.fits")

esmcmc.set_nthreads (4)
t0 = time.time()
esmcmc.start_run ()
esmcmc.run (ssize / nwalkers)
#esmcmc.run (10)
#esmcmc.run_lre (50, 1.0e-3)
esmcmc.end_run ()
t1 = time.time()

esmcmc.mean_covar ()
fit.log_covar ()

print('Full sky maximal time (seconds) = ' + str(t1-t0))

ncounts_gauss.peek_mean().log_vals("y: ", "%.3f", True)
ncounts_gauss.peek_cov().cov_dup_cor().log_vals("cov:", "%.3f")

Partial Sky maximal area
y: 85379.339 204030.586 344022.259 490554.538 615158.399 720801.528 783103.137
0.8470865810460315
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensemble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0004.
#   Using MPI:         no - use MPI enabled but no slaves available.
# NcmFitESMCMC: No RNG was defined, using algorithm: `mt19937' and seed: 2890341744.
# NcmMSetCatalog: Current mean:   2.2357e+05   0.99436      0.25477     -1.0312       3.29       
# NcmMSetCatalog: Current msd:    28881        0.016305     0.00045702   0.00044876   0.031614   
# NcmMSetCatalog: Current sd:     6.458e+05    0.3646       0.010219     0.010035     0.7069     
# NcmMSetCatalog: Current var:    4.1706e+11   0.13293      0.00010443   0.00010069   0.49971    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Curre


(process:72054): NUMCOSMO-WARNING **: 20:49:18.779: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   9130.8       0.86338      0.25534     -1.0201       3.1323     
# NcmMSetCatalog: Current msd:    990.78       0.00077183   8.3174e-05   0.00027181   0.0018381  
# NcmMSetCatalog: Current sd:     1.2335e+05   0.096092     0.010355     0.03384      0.22884    
# NcmMSetCatalog: Current var:    1.5215e+10   0.0092336    0.00010723   0.0011452    0.052367   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99921      1.0274       1.2196       1.2849       1.1013     
# NcmMSetCatalog: Maximal Shrink factor = 1.28853               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `31':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 15.7333% (last update  3.6000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCM


(process:72054): NUMCOSMO-WARNING **: 20:49:46.698: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   8589.3       0.86231      0.25548     -1.02         3.1295     
# NcmMSetCatalog: Current msd:    930.88       0.00072787   8.0441e-05   0.00026992   0.0017491  
# NcmMSetCatalog: Current sd:     1.1957e+05   0.093496     0.010333     0.034672     0.22468    
# NcmMSetCatalog: Current var:    1.4298e+10   0.0087415    0.00010677   0.0012022    0.05048    
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99926      1.0274       1.2156       1.2932       1.1014     
# NcmMSetCatalog: Maximal Shrink factor = 1.29508               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `33':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 14.9375% (last update  2.6000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCM


(process:72054): NUMCOSMO-WARNING **: 20:57:16.716: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   6212.2       0.85793      0.25594     -1.0199       3.1184     
# NcmMSetCatalog: Current msd:    668.26       0.00053145   6.531e-05    0.00026081   0.0013286  
# NcmMSetCatalog: Current sd:     1.0135e+05   0.080598     0.0099047    0.039554     0.20149    
# NcmMSetCatalog: Current var:    1.0271e+10   0.006496     9.8104e-05   0.0015645    0.040599   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99946      1.0268       1.1966       1.2807       1.0964     
# NcmMSetCatalog: Maximal Shrink factor = 1.28211               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `46':
# NcmMSetCatalog: Constant break point at `6':
# NcmFitESMCMC:acceptance ratio 12.6978% (last update  4.2000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMCM


(process:72054): NUMCOSMO-WARNING **: 21:26:27.549: _ncm_data_gauss_cov_prepare_LLT[ncm_matrix_cholesky_decomp]: 1.


# NcmMSetCatalog: Current mean:   2930.9       0.85106      0.25737     -1.0165       3.0988     
# NcmMSetCatalog: Current msd:    307.7        0.00024945   3.506e-05    0.00021384   0.00065136 
# NcmMSetCatalog: Current sd:     68803        0.055778     0.0078396    0.047815     0.14565    
# NcmMSetCatalog: Current var:    4.7338e+09   0.0031112    6.1459e-05   0.0022863    0.021214   
# NcmMSetCatalog: Current tau:    1            1            1            1            1          
# NcmMSetCatalog: Current skfac:  0.99974      1.0131       1.0751       1.0705       1.0432     
# NcmMSetCatalog: Maximal Shrink factor = 1.07671               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `100':
# NcmMSetCatalog: Constant break point at `9':
# NcmFitESMCMC:acceptance ratio 20.0990% (last update 39.2000%), offboard ratio  0.0000% (last update  0.0000%).
# NcmFitESMC